# Text Augmentation


1. Easy Data Augmentation
    - Random Deletion (직접 구현해보기)
    - Random Insertion
    - Random Swap (직접 구현해보기)
    - Synonym Replacement (동의어 replace)
2. Back Translation (직접 구현해보기)

In [1]:
import pandas as pd
import random
import pickle
import re

In [10]:
import gdown
 
url = 'https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS'
output = 'spam.txt'
gdown.download(url, output, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS
From (redirected): https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS&confirm=t&uuid=bc267b68-f618-4555-bcdb-9a0005a936d5
To: c:\code\skkukdt_minzy\DeepLearning\spam.txt
100%|██████████| 800M/800M [00:25<00:00, 31.4MB/s]


'spam.txt'

In [2]:
!gdown https://drive.google.com/uc?id=1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS

'gdown'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [11]:
df_news = pd.read_json("./train_original.json")
df_news

 18%|█▊        | 144M/800M [10:42<48:56, 224kB/s] 


,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"
...,...,...,...,...,...
260692,매일경제,353316985,"[두통과 어지럼증은 우리에게 매우 익숙하다., 하루 이틀 정도만 참으면 자연스럽게 ...","두통과 어지럼증은 우리에게 매우 익숙한 증상이며, 이 두통은 크게 긴장성 두통, 혈...","[3, 12, 16]"
260693,부산일보,360784906,[파주 최대 테마파크 퍼스트가든이 ㈜투바앤의 캐릭터 '라바'와 할로윈 페스티벌(사진...,파주 최대 테마파크 퍼스트가든이 오는 10월 12일부터 31일까지 가족들이 함께 즐...,"[0, 4, 1]"
260694,부산일보,357253753,[민원과 난공사로 완공이 늦어지고 있는 양산시 한일아파트 인근 옹벽 보수·보강공사 ...,양산시는 지난해 10월부터 43억 원을 들여 진행 중인 한일아파트 뒤편 옹벽 보수·...,"[4, 6, 7]"
260695,매일신문,347655953,"[사단법인 전국자원봉사연맹이 호국보훈의 달을 맞이하여, 오는 2019년 6월 23일...","국가를 위해 헌신한 분들의 숭고한 희생을 기리고, 유가족 들을 위로하기 위해 열리는...","[0, 1, 2]"


In [12]:
train = df_news[['abstractive']].loc[:100]
train.head()

,abstractive
0,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...
1,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...
2,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...
3,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...
4,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...


In [13]:
sample = train.values[0, 0]
sample

"통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. "

### Random Deletion

< 사용할 함수 >   
- random.uniform(0, 1)
- random.randint(0, n)

In [ ]:
# 정한 확률만큼 deletion
def random_deletion(sent, p):
    # 길이가 1이면 안지운다.
	pass

### Random Swap

In [4]:
# 정한 갯수만큼 swap
def random_swap(sent, n):
    pass

## Back Translation

In [6]:
!pip install --upgrade translators

  Obtaining dependency information for translators from https://files.pythonhosted.org/packages/3c/62/e95f09c6cc425f002827acd10fbeb06c67d237142831cf3488cf1e2ea2cf/translators-5.8.3-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/68.5 kB ? eta -:--:--
     ---------------------------------------- 68.5/68.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for pathos>=0.2.9 from https://files.pythonhosted.org/packages/d8/08/ac94fa6f9eefe32963b8a54e573dab0dbc0d3df24fd34924bd9ce7eab7c4/pathos-0.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for ppft>=1.7.6.7 from https://files.pythonhosted.org/packages/f0/f8/0a493dfdf73edbfe58cae1323aec72d0152f463c7a351bd285e9d500985c/ppft-1.7.6.7-py3-none-any.whl.metadata
  Obtaining dependency information for pox>=0.3.3 from https://files.pythonhosted.org/packages/17/c7/ef7e37e5a895f5de068b408a52bee0

 18%|█▊        | 144M/800M [02:40<01:32, 7.13MB/s]

In [7]:
import translators as ts

Using region Gyeonggi-do server backend.



In [8]:
print(ts.translators_pool)

['alibaba', 'apertium', 'argos', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'deepl', 'elia', 'google', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 'judic', 'languageWire', 'lingvanex', 'niutrans', 'mglip', 'mirai', 'modernMt', 'myMemory', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 'translateCom', 'translateMe', 'utibet', 'volcEngine', 'yandex', 'yeekit', 'youdao']


In [14]:
sample

"통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계를 작성한 이래 가장 많았으며 사망 원인 중 폐렴과 알츠하이머 병의 순위 상승이 두드러졌고 자살에 의한 사망자도 증가했다. "

In [16]:
sample_eng = ts.translate_text(sample, translator='baidu')
sample_eng

'According to the "2018 Cause of Death Statistics" released by the Department of Statistics, the total number of deaths last year was the highest since the establishment of relevant statistics. Among the causes of death, the ranking of pneumonia and Alzheimer\'s disease has increased significantly, and the number of suicide deaths has also increased.'

In [29]:
sample_aug = ts.translate_text(sample_eng, to_language='ko')
sample_aug

"통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병이 순위에 올랐고 자살로 인한 사망자도 증가했다."

### Implementation

1. translator 간의 성능 비교
2. 활용 언어 간의 성능 비교
3. 위 샘플 텍스트 뭉치를 5배로 증강해보기

In [29]:
# 파파고가 나은거같다...
print(ts.translate_text(sample, translator='alibaba', to_language='en', from_language='ko'))
print(ts.translate_text(sample, translator='google', to_language='en', from_language='ko'))
print(ts.translate_text(sample, translator='papago', to_language='en', from_language='ko'))
print(ts.translate_text(sample, translator='bing', to_language='en', from_language='ko'))


According to the 2018 cause of death statistics released by the National Statistics Office, the total number of deaths last year was the highest since the relevant statistics were prepared, followed by pneumonia and Alzheimer&#39;s disease among the causes of death. The rise in stomach was prominent, and the number of deaths by suicide also increased.
According to the 2018 Causes of Death, the total number of deaths was the largest since last year's statistics.
According to the "2018 Statistics on the Cause of Death" released by the National Statistical Office, the total number of deaths last year was the highest since related statistics were compiled, with the rise in the ranking of pneumonia and Alzheimer's disease among the causes of death and suicide.
According to the "2018 Cause of Death Statistics" released by the National Statistics Office, the total number of deaths last year was the highest since the relevant statistics were compiled, and among the causes of death, pneumonia a

In [32]:
print(ts.get_languages('papago')['ko'])

['de', 'en', 'es', 'fr', 'hi', 'id', 'it', 'ja', 'ko', 'pt', 'ru', 'th', 'vi', 'zh-CN', 'zh-TW']


In [36]:
import time
for lang in ts.get_languages('papago')['ko']:
    if lang == 'ko':
        continue
    toOther = ts.translate_text(sample, translator='papago', to_language=lang, from_language='ko')
    print(lang + ' : ' + ts.translate_text(toOther, translator='papago', to_language='ko', from_language=lang))
    time.sleep(3)

de : 2018년 사망원인에 대한 통계는 지난해 전체 사망자 수가 통계 작성 이후 가장 많았음을 보여준다.
en : 통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 전체 사망자 수는 사망·자살 원인 중 폐렴·알츠하이머병의 순위가 상승하는 등 관련 통계 작성 이래 가장 많았다.
es : 국립통계연구소가 발표한 2018년 사망 원인 통계는 지난해 관련 통계가 집계된 이후 가장 많은 사망자가 발생했으며 자살로 인한 사망 원인 중 폐렴과 알츠하이머 발병률이 증가했다.
fr : 통계청이 발표한 2018년 통계에 따르면 전체 사망자는 지난해 통계 작성 이후 가장 많았으며 사망 원인 중 폐렴과 알츠하이머병이 증가했다.
hi : 통계청이 발표한 '2018년 사망자 통계'에 따르면 관련 자료 수집 
id : 통계청이 발표한 '2018년 사망원인통계'에 따르면 지난해 전체 사망자는 관련 통계가 작성된 이후 가장 많았고, 사망 원인 중 폐렴과 알츠하이머병 순위는 증가했다.
it : 국립통계청이 발표한 2018년 사망원인에 대한 통계에 따르면, 작년에 폐렴과 알츠하이머의 수가 증가하면서 총 사망자가 가장 많았고 자살로 인한 사망자 수가 가장 많았다.
ja : 통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 총 사망자 수는 관련 통계 작성 이래 가장 많았고 사망원인 중 폐렴과 알츠하이머병 순위 상승이 두드러졌으며 자살로 인한 사망자도 증가했다.
pt : 통계청이 발간한 '사망통계 사례'에 따르면 지난해 전체 사망자 수는 여성의 폐렴과 알츠하이머병 분류가 증가하는 등 관련 통계 작성 이래 가장 많았다. 
ru : 통계청이 발표한 2018년 통계에 따르면 지난해 전체 사망자는 관련 통계를 작성한 이후 가장 많았다. 사망 원인 중에는 폐렴과 알츠하이머 병의 순위가 눈에 띄게 상승했고 자살로 인한 사망자가 증가했다.
th : 통계청이 발표한 '2018년 사망원인 통계'에 따르면 지난해 전체 사망자는 관련 통계가 작성된 이후 가장 많았고 사망원인 폐렴과 알츠하이머병

In [38]:
from tqdm.notebook import tqdm
train_5 = []
for t in tqdm(train['abstractive']):
    toKor = t
    for _ in range(5):
        toOther = ts.translate_text(toKor, translator='papago', to_language='zh-CN', from_language='ko')
        toKor = ts.translate_text(toOther, translator='papago', to_language='ko', from_language='zh-CN')
        train_5.append(toKor)
        

  0%|          | 0/101 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [40]:
len(train_5)

69

In [41]:
train_5

["통계청이 발표한 '2018년 사망원인통계'에 따르면 지난해 총 사망자 수는 관련 통계 작성 이후 최대치로, 사망 원인 중 폐렴과 알츠하이머병의 순위 상승이 두드러졌고 자살로 인한 사망자 수도 증가했다.",
 "통계청이 발표한 '2018년 사망원인통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이래 최고치를 기록했고, 사망 원인 중 폐렴과 알츠하이머병의 순위 상승이 두드러지며 자살로 인한 사망자 수도 증가했다.",
 "통계청이 발표한 '2018년 사망원인통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이래 최고치를 기록했고, 사망 원인 중 폐렴과 알츠하이머병의 순위 상승이 두드러지며 자살로 인한 사망자 수도 증가했다.",
 "통계청이 발표한 '2018년 사망원인통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이래 최고치를 기록했고, 사망 원인 중 폐렴과 알츠하이머병의 순위 상승이 두드러지며 자살로 인한 사망자 수도 증가했다.",
 "통계청이 발표한 '2018년 사망원인통계'에 따르면 지난해 총 사망자 수는 관련 통계 집계 이래 최고치를 기록했고, 사망 원인 중 폐렴과 알츠하이머병의 순위 상승이 두드러지며 자살로 인한 사망자 수도 증가했다.",
 '서산시의원 가충순과 이수의 의원은 적극적인 의정활동에 대한 감사의 표시로 한국지역신문협회가 수여하는 우수 의정대상을 수상했으며, 가충수 의원은 자동차연비 테스트 연구시설 유치, 천수만 염해 재발방지, 서산시 대표농산물 육성 등 지역인재 채용 확대 및 관내 기업자재 활용을 위한 다양한 의정활동을 제안했다.',
 '서산시의원인 가충순과 이수의 의원은 적극적인 의정활동에 대한 감사의 표시로 한국지역신문협회가 수여하는 우수 의정대상을 수상했으며, 가충수 의원은 자동차연비 테스트 연구시설 유치, 천수만 염해 재발방지, 서산시 대표농산물 육성 등 지역인재 채용 확대 및 관내 기업소재를 활용한 다양한 의정활동을 제안했다.',
 '서산시의원인 가충순과 이수의 의원은 적극적인 의정활동에 감사해 한국지역신문협회가